In [1]:
%env CUDA_VISIBLE_DEVICES=5

env: CUDA_VISIBLE_DEVICES=5


In [2]:
import numpy as np
import cupy as cp

In [3]:
xp = np

X = 3
Y = 3
Z = 2
K = 1
data = np.random.rand(X,Y,Z).astype('float32')

# numpy

In [4]:
def vectors_comparison(vector1, vector2, data_stds, shifted_stds):
    res = xp.multiply(vector1, vector2)
    res = xp.sum(xp.power(res, 2), axis = 2)
#     res = xp.multiply(res, data_stds)
#     res = xp.multiply(res, shifted_stds)
    return res

In [5]:
data = data = np.random.randint(0, 2, (X,Y,Z)).astype('float32')
bad_traces=np.zeros(data.shape[:2])
# check with bad traces

In [6]:
data_stds = data.std(axis=-1)
tmp_bad_traces = bad_traces.copy()
tmp_bad_traces[data_stds == 0.0] = 1

In [7]:
np.any(tmp_bad_traces==1)

True

In [8]:
np.all(tmp_bad_traces==1)

False

# old

In [9]:
def compute_local(function, data, bad_traces, kernel_size=3,
                      normalize=True, agg='mean', amortize=False, axis=0, device='cpu', pbar=None):
        i_range, x_range = data.shape[:2]
        k = kernel_size // 2

        # Transfer to GPU, if needed
        data = to_device(data, device)
        bad_traces = to_device(bad_traces, device)
        xp = cp.get_array_module(data) if (cp is not np) else np

        # Compute data statistics
        data_stds = data.std(axis=-1)
        bad_traces[data_stds == 0.0] = 1
        if normalize:
            data_n = data - data.mean(axis=-1, keepdims=True)
        else:
            data_n = data

        # Pad everything
        padded_data = xp.pad(data_n, ((k, k), (k, k), (0, 0)), constant_values=xp.nan)
        padded_stds = xp.pad(data_stds, k, constant_values=0.0)
        padded_bad_traces = xp.pad(bad_traces, k, constant_values=1)
        
        # Compute metric by shifting arrays
        total = kernel_size * kernel_size - 1
        pbar = Notifier(pbar, total=total) if pbar else None

        accumulator = Accumulator(agg=agg, amortize=amortize, axis=axis, total=total)
        for i in range(kernel_size):
            for j in range(kernel_size):
                if i == j == k:
                    continue

                shifted_data = padded_data[i:i+i_range, j:j+x_range]
                shifted_stds = padded_stds[i:i+i_range, j:j+x_range]
                shifted_bad_traces = padded_bad_traces[i:i+i_range, j:j+x_range]

                computed = function(data_n, shifted_data, data_stds, shifted_stds)
                computed[shifted_bad_traces == 1] = xp.nan

                accumulator.update(computed)
                if pbar:
                    pbar.update()
        if pbar:
            pbar.close()

        result = accumulator.get(final=True)
        return from_device(result)

In [10]:
import sys
sys.path.append("/notebooks") 
sys.path.append("..")

In [11]:
from seismiqb.src.utility_classes import Accumulator
from seismiqb.src.functional import to_device, from_device
from batchflow.notifier import Notifier

In [12]:
data_n = data

In [13]:
# %%timeit -o
res = compute_local(function=vectors_comparison, data=data_n.copy(), bad_traces=bad_traces.copy(), kernel_size=3,
                      normalize=False, agg='nanmean', amortize=False, axis=0, device='cpu', pbar=True)

100%|██████████████████████████████████████████| 8/8 [00:00<00:00, 11852.50it/s]
/notebooks/seismiqb/seismiqb/src/utility_classes.py:197: RuntimeWarning:

Mean of empty slice



In [14]:
# times = _
# timings = times.timings

# new

In [32]:
def compute_local_new(function, data, bad_traces, kernel_size=3,
                      normalize=True, agg='mean', amortize=False, axis=0, device='cpu', pbar=None):
        i_range, x_range = data.shape[:2]
        k = kernel_size // 2 + 1

        # Transfer to GPU, if needed
        data = to_device(data, device)
        bad_traces = to_device(bad_traces, device)
        xp = cp.get_array_module(data) if (cp is not np) else np

        # Compute data statistics
        data_stds = data.std(axis=-1)
        bad_traces[data_stds == 0.0] = 1
        if normalize:
            data_n = data - data.mean(axis=-1, keepdims=True)
        else:
            data_n = data

        # Pad everything
        padded_data = xp.pad(data_n, ((0, k), (k, k), (0, 0)), constant_values=xp.nan)
        padded_stds = xp.pad(data_stds, ((0, k), (k, k)), constant_values=0.0)
        padded_bad_traces = xp.pad(bad_traces, ((0, k), (k, k)), constant_values=1)

        # Compute metric by shifting arrays
        total = kernel_size * kernel_size - 1
        pbar = Notifier(pbar, total=total//2) if pbar else None

        accumulator = Accumulator(agg=agg, amortize=amortize, axis=axis, total=total)
                        
        for i in range(k):
            for j in range(-k+1, k):
                if (i == 0) and (j <= 0):
                    continue                    
                shifted_data = padded_data[i:i+i_range, k+j:k+j+x_range]
                shifted_stds = padded_stds[i:i+i_range, k+j:k+j+x_range]
                shifted_bad_traces = padded_bad_traces[i:i+i_range, k+j:k+j+x_range]

                # Compute metric value for traces (x, y) and (x+i, y+j)
                computed = function(data, shifted_data, data_stds, shifted_stds)                

                # Shift comparison matrix and add padding for traces without pairs for comparison
                splitted_computed = computed[:i_range-i, max(0, -j):min(x_range, x_range-j)]
                shifted_computed = xp.pad(splitted_computed,
                                          ((i, 0), (max(0, j), -min(0, j))), 
                                          constant_values=xp.nan
                                         )
                
                computed[shifted_bad_traces == 1] = xp.nan                
                
#                 splitted_bad_traces = shifted_bad_traces[:i_range-i, max(0, -j):min(x_range, x_range-j)]
#                 shifted_bad_traces = xp.pad(splitted_bad_traces,
#                                           ((i, 0), (max(0, j), -min(0, j))), 
#                                           constant_values=1
#                                          )
#                 shifted_bad_traces_2 = padded_bad_traces[:i_range, k:k+x_range]
#                 shifted_computed[shifted_bad_traces_2 == 1] = xp.nan

                accumulator.update(computed)
                accumulator.update(shifted_computed)
                if pbar:
                    pbar.update()

        if pbar:
            pbar.close()

        result = accumulator.get(final=True)
        return from_device(result)

In [33]:
res2 = compute_local_new(function=vectors_comparison, data=data_n.copy(), bad_traces=bad_traces.copy(), kernel_size=3,
                      normalize=False, agg='nanmean', amortize=False, axis=0, device='cpu', pbar=True)


100%|███████████████████████████████████████████| 4/4 [00:00<00:00, 2547.41it/s]


In [34]:
np.allclose(res, res2, equal_nan=True)

False

In [35]:
res

array([[0.5, 0.5, 1. ],
       [0. , 0. , 1. ],
       [0. , 0. , nan]])

In [36]:
res2

array([[0.5       , 0.5       ,        nan],
       [0.        , 0.        , 1.        ],
       [       nan, 0.        , 0.33333333]])

In [20]:
tmp_bad_traces

array([[0., 0., 1.],
       [0., 1., 1.],
       [1., 1., 0.]])